In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore")

In [0]:
customers = pd.read_csv("/Workspace/Users/kathalenikhil741@gmail.com/Customers.csv")
engagement = pd.read_csv("/Workspace/Users/kathalenikhil741@gmail.com/Engagement.csv")
portfolio = pd.read_csv("/Workspace/Users/kathalenikhil741@gmail.com/Portfolio.csv")
support = pd.read_csv("/Workspace/Users/kathalenikhil741@gmail.com/Support.csv")
trading = pd.read_csv("/Workspace/Users/kathalenikhil741@gmail.com/TradingActivity.csv")

##DATA OVERVIEW 

In [0]:
print("Customers:", customers.shape)
print("Engagement:", engagement.shape)
print("Portfolio:", portfolio.shape)
print("Support:", support.shape)
print("TradingActivity:", trading.shape)


Customers: (50000, 6)
Engagement: (50000, 5)
Portfolio: (145994, 5)
Support: (50000, 5)
TradingActivity: (200000, 7)


In [0]:
print("Customers:", customers.info)
print("Engagement:", engagement.info)
print("Portfolio:", portfolio.info)
print("Support:", support.info)
print("TradingActivity:", trading.info)

Customers: <bound method DataFrame.info of                                 customer_id  ... account_open_date
0      fd94fe1f-8eb2-4623-aa3c-42040b84dad9  ...        31-12-2023
1      748ad9c4-43c9-40e7-ad3d-7173827abe5e  ...        05-10-2020
2      0cdbb69c-14c0-46e6-aaae-63ac632eceea  ...        07-05-2024
3      940c4ee2-1b31-4c61-942b-fe6256ddb797  ...        17-09-2020
4      b98d8294-d450-4b5b-bcc8-dce9587ebea5  ...        12-07-2021
...                                     ...  ...               ...
49995  e423b767-6e69-463c-8be3-44cdb8c1a835  ...        12-12-2023
49996  c7c57773-f556-44da-948f-08d771cc25cd  ...        16-09-2022
49997  5c3a6120-c325-471f-b86b-6d1148ec3321  ...        17-12-2023
49998  dfd653ee-631c-4cc7-95b6-52749576175e  ...        26-05-2024
49999  5159e48f-f81b-4cb5-8ef1-d096094e62ea  ...        30-06-2023

[50000 rows x 6 columns]>
Engagement: <bound method DataFrame.info of                                 customer_id  ...  last_active_date
0      fd94fe1f

In [0]:
print(customers.head(2))
print(engagement.head(2))
print(portfolio.head(2))
print(support.head(2))
print(trading.head(2))

                            customer_id  ... account_open_date
0  fd94fe1f-8eb2-4623-aa3c-42040b84dad9  ...        31-12-2023
1  748ad9c4-43c9-40e7-ad3d-7173827abe5e  ...        05-10-2020

[2 rows x 6 columns]
                            customer_id  ...  last_active_date
0  fd94fe1f-8eb2-4623-aa3c-42040b84dad9  ...        24-08-2025
1  748ad9c4-43c9-40e7-ad3d-7173827abe5e  ...        24-08-2025

[2 rows x 5 columns]
                            customer_id  ...                          portfolio_id
0  00001773-c1a6-4652-8ba9-dc6eefcd4cd1  ...  806071e6-2115-4927-8869-cbeea477f087
1  00001773-c1a6-4652-8ba9-dc6eefcd4cd1  ...  2ec611cd-f731-4e4b-b86d-332245fa444e

[2 rows x 5 columns]
                              ticket_id  ... resolved
0  23671ed5-9cf6-4a6a-85f9-0fab1aac6657  ...        1
1  d45fe7eb-4a1f-42be-8bc1-81a539e6f59f  ...        1

[2 rows x 5 columns]
                               trade_id  ...  trade_date
0  b74a429f-4701-467e-9329-4d3573ab4c28  ...  07-04-2024
1  84c2ca

##MERGING THE REQUIRED DATASET 

In [0]:
for customer_data in [engagement, portfolio, support, trading]:
    if "customer_id" not in customer_data.columns:
        customer_data.rename(columns={customer_data.columns[0]: "customer_id"}, inplace=True)

#merging all the dataset
customer_data = customers.merge(engagement, on="customer_id", how="left")\
              .merge(portfolio, on="customer_id", how="left")\
              .merge(support, on="customer_id", how="left")\
              .merge(trading, on="customer_id", how="left")

print("Final merged dataset:", customer_data.shape)

Final merged dataset: (937590, 24)


##LETS DO THE EDA

In [0]:
missing = customer_data.isnull().mean().sort_values(ascending=False)
missing

resolution_time_hours    0.268474
resolved                 0.268474
issue_type               0.268474
ticket_id                0.268474
stock_symbol_x           0.001376
holding_volume           0.001376
avg_buy_price            0.001376
portfolio_id             0.001376
trade_type               0.001376
trade_volume             0.001376
trade_id                 0.001376
stock_symbol_y           0.001376
trade_price              0.001376
trade_date               0.001376
name                     0.000000
customer_id              0.000000
watchlist_count          0.000000
app_logins               0.000000
account_open_date        0.000000
account_type             0.000000
location                 0.000000
age                      0.000000
last_active_date         0.000000
avg_session_minutes      0.000000
dtype: float64

##PARSE DATES 


In [0]:
for customer_data, cols in [
    (customers,  ["account_open_date"]),
    (engagement, ["last_active_date"]),
    (trading,    ["trade_date"]),
]:
    for c in cols:
        if c in customer_data.columns:
            customer_data[c] = pd.to_datetime(customer_data[c], errors="coerce")

## REFRENCE "as-of" Dates 

In [0]:
max_trade_date = trading["trade_date"].max() if "trade_date" in trading else pd.NaT
max_active_date = engagement["last_active_date"].max() if "last_active_date" in engagement else pd.NaT
as_of_date = max(d for d in [max_trade_date, max_active_date] if pd.notna(d)) \
             if any(pd.notna(d) for d in [max_trade_date, max_active_date]) else pd.Timestamp.today().normalize()

In [0]:
print(max_trade_date)
print(max_active_date)
print(as_of_date)

2025-12-08 00:00:00
2025-08-24 00:00:00
2025-12-08 00:00:00


##TRADING FEATURES ( as PER CUSTOMER)

In [0]:
if not trading.empty:
    trading_feat = trading.copy()
    trading_feat["trade_value"] = trading_feat["trade_volume"] * trading_feat["trade_price"]

    grp = trading_feat.groupby("customer_id")
    trading_agg = pd.DataFrame({
        "trade_count": grp.size(),
        "total_trade_volume": grp["trade_volume"].sum(),
        "total_trade_value": grp["trade_value"].sum(),
        "avg_trade_value": grp["trade_value"].mean(),
        "avg_trade_price": grp["trade_price"].mean(),
        "symbol_diversity": grp["stock_symbol"].nunique(),
        "active_trade_days": grp["trade_date"].nunique(),
        "last_trade_date": grp["trade_date"].max()
    }).reset_index()

    # BUY/SELL split
    buys = (trading_feat["trade_type"] == "BUY").groupby(trading_feat["customer_id"]).sum().rename("buy_trades")
    sells = (trading_feat["trade_type"] == "SELL").groupby(trading_feat["customer_id"]).sum().rename("sell_trades")
    trading_agg = trading_agg.merge(buys, on="customer_id", how="left").merge(sells, on="customer_id", how="left").fillna(0)
    trading_agg["buy_sell_ratio"] = trading_agg["buy_trades"] / (trading_agg["sell_trades"])

    # Recency
    trading_agg["days_since_last_trade"] = (as_of_date - trading_agg["last_trade_date"]).dt.days
    trading_agg.drop(columns=["last_trade_date"], inplace=True)
else:
    trading_agg = pd.DataFrame(columns=[
        "customer_id","trade_count","total_trade_volume","total_trade_value","avg_trade_value",
        "avg_trade_price","symbol_diversity","active_trade_days","buy_trades","sell_trades",
        "buy_sell_ratio","days_since_last_trade"
    ])

In [0]:
trading_agg

,customer_id,trade_count,total_trade_volume,total_trade_value,avg_trade_value,avg_trade_price,symbol_diversity,active_trade_days,buy_trades,sell_trades,buy_sell_ratio,days_since_last_trade
0,00001773-c1a6-4652-8ba9-dc6eefcd4cd1,5,1400,3901080.98,780216.196000,2673.096000,4,5,4,1,4.00,537
1,00011eab-8526-47f2-a39e-f3a4c2ee151c,3,665,1158148.90,386049.633333,1757.340000,3,3,2,1,2.00,135
2,0001afd9-6f07-4201-a477-b5d31fb0cc21,2,540,266002.77,133001.385000,858.055000,2,2,1,1,1.00,746
3,0001b133-92b6-41c1-b884-92e6cba788b2,3,404,450895.50,150298.500000,1652.500000,3,3,2,1,2.00,447
4,00054a90-3ad3-4b29-ba22-dc34707d702c,7,1680,3117267.49,445323.927143,2002.307143,5,7,3,4,0.75,30
...,...,...,...,...,...,...,...,...,...,...,...,...
49078,fff8cbd2-051c-4e4a-9e40-a23dfb964e77,6,1715,2704973.18,450828.863333,1464.301667,4,6,4,2,2.00,384
49079,fffb8be6-92e8-4017-8062-c9a597d464e0,2,710,785011.80,392505.900000,1039.260000,1,2,2,0,inf,444
49080,fffbac30-4a3f-4beb-999e-cb3af77a0bc5,5,1073,1080131.08,216026.216000,1118.586000,4,5,3,2,1.50,142
49081,fffd9e02-58a5-4808-86fa-1e13399b6c29,3,811,1452540.46,484180.153333,1628.813333,3,3,2,1,2.00,124


## PORTFOLIO FEATURES by CUSTOMERS

In [0]:
if not portfolio.empty:
    pf = portfolio.copy()
    pf["position_value"] = pf["holding_volume"] * pf["avg_buy_price"]

    # weights per symbol -> HHI concentration & largest position %
    def _portfolio_concentration(df):
        total = df["position_value"].sum()
        if total <= 0:
            return pd.Series({
                "portfolio_value": 0.0,
                "positions_count": df["stock_symbol"].nunique(),
                "largest_position_pct": 0.0,
                "hhi_concentration": 0.0
            })
        w = df["position_value"] / total
        hhi = (w**2).sum()
        return pd.Series({
            "portfolio_value": total,
            "positions_count": df["stock_symbol"].nunique(),
            "largest_position_pct": w.max(),
            "hhi_concentration": hhi
        })

    portfolio_agg = pf.groupby("customer_id").apply(_portfolio_concentration).reset_index()
else:
    portfolio_agg = pd.DataFrame(columns=[
        "customer_id","portfolio_value","positions_count","largest_position_pct","hhi_concentration"
    ])

In [0]:
portfolio_agg

,customer_id,portfolio_value,positions_count,largest_position_pct,hhi_concentration
0,00001773-c1a6-4652-8ba9-dc6eefcd4cd1,3.918697e+06,4.0,0.414751,0.298932
1,00011eab-8526-47f2-a39e-f3a4c2ee151c,1.158149e+06,3.0,0.811167,0.676122
2,0001afd9-6f07-4201-a477-b5d31fb0cc21,2.660028e+05,2.0,0.591738,0.516832
3,0001b133-92b6-41c1-b884-92e6cba788b2,4.508955e+05,3.0,0.700560,0.553199
4,00054a90-3ad3-4b29-ba22-dc34707d702c,3.380797e+06,5.0,0.310853,0.253726
...,...,...,...,...,...
49078,fff8cbd2-051c-4e4a-9e40-a23dfb964e77,2.707730e+06,4.0,0.726730,0.567761
49079,fffb8be6-92e8-4017-8062-c9a597d464e0,7.378746e+05,1.0,1.000000,1.000000
49080,fffbac30-4a3f-4beb-999e-cb3af77a0bc5,1.090431e+06,4.0,0.458325,0.410006
49081,fffd9e02-58a5-4808-86fa-1e13399b6c29,1.452540e+06,3.0,0.606376,0.495389


##ENGAMENT FEATURES by CUSOMTER
to determine  

In [0]:
if not engagement.empty:
    eng = engagement.copy()
    eng["days_since_last_active"] = (as_of_date - eng["last_active_date"]).dt.days
    engagement_agg = eng[[
        "customer_id","app_logins","watchlist_count","avg_session_minutes","days_since_last_active"
    ]].copy()
else:
    engagement_agg = pd.DataFrame(columns=[
        "customer_id","app_logins","watchlist_count","avg_session_minutes","days_since_last_active"
    ])

In [0]:
engagement_agg

,customer_id,app_logins,watchlist_count,avg_session_minutes,days_since_last_active
0,fd94fe1f-8eb2-4623-aa3c-42040b84dad9,323,30,49.92,106
1,748ad9c4-43c9-40e7-ad3d-7173827abe5e,60,22,30.78,106
2,0cdbb69c-14c0-46e6-aaae-63ac632eceea,464,18,3.57,106
3,940c4ee2-1b31-4c61-942b-fe6256ddb797,435,43,56.72,106
4,b98d8294-d450-4b5b-bcc8-dce9587ebea5,295,39,11.40,106
...,...,...,...,...,...
49995,e423b767-6e69-463c-8be3-44cdb8c1a835,315,20,50.19,106
49996,c7c57773-f556-44da-948f-08d771cc25cd,337,13,51.61,106
49997,5c3a6120-c325-471f-b86b-6d1148ec3321,73,33,58.63,106
49998,dfd653ee-631c-4cc7-95b6-52749576175e,51,39,45.43,106


##CUSTOMER SUPPORT RESOLUTION 

In [0]:
if not support.empty:
    sup = support.copy()
    
    # Group by customer to calculate support-related metrics
    sup_grp = sup.groupby("customer_id")
    
    # Aggregate metrics:
    # - tickets_count: total tickets per customer
    # - avg_resolution_time_hours: mean time to resolve tickets
    # - unresolved_rate: share of unresolved tickets (1 - mean(resolved))
    support_agg = pd.DataFrame({
        "customer_id": sup_grp.size().index,  # customer_id after grouping
        "tickets_count": sup_grp.size().values,  
        "avg_resolution_time_hours": sup_grp["resolution_time_hours"].mean().values,
        "unresolved_rate": (1 - sup_grp["resolved"].mean()).values
    }).reset_index(drop=True)
    
else:
    # In case no support data is available
    support_agg = pd.DataFrame(columns=[
        "customer_id","tickets_count","avg_resolution_time_hours","unresolved_rate"
    ])

In [0]:
support_agg

,customer_id,tickets_count,avg_resolution_time_hours,unresolved_rate
0,00011eab-8526-47f2-a39e-f3a4c2ee151c,1,70.0,1.0
1,0001afd9-6f07-4201-a477-b5d31fb0cc21,2,55.5,0.0
2,00054a90-3ad3-4b29-ba22-dc34707d702c,2,24.5,0.0
3,0006fada-cc42-475c-9e32-d06513ca1562,1,59.0,0.0
4,00071cd6-bf02-4eb1-b954-713198d8b0e6,1,7.0,0.0
...,...,...,...,...
31626,fff5985a-19dc-4450-861d-b2ec80c713e1,1,24.0,0.0
31627,fff6fcb3-43e3-4e5f-a962-409a1bbdb6a0,2,32.5,0.0
31628,fff8cbd2-051c-4e4a-9e40-a23dfb964e77,1,20.0,0.0
31629,fffb8be6-92e8-4017-8062-c9a597d464e0,1,27.0,0.0
